In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import json
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import string


In [3]:
with open('sample.json', 'r',encoding='utf-8') as file:
    sentences = json.load(file)
sentences = [elem['text'] for elem in sentences]
sentences

['Ты нашёл их или нет?',
 'Почему она так со мной поступает?',
 'Никто туда больше не ходит.',
 'У него с собой не было тогда денег.',
 'Почему они с нами так поступают?',
 'Он всю ночь стонал от сильной боли.',
 'Я больше не хочу с тобой играть.',
 'Тому было тогда всего тринадцать лет.',
 'Что сделал Том с деньгами?',
 'Том меня сейчас хочет видеть?',
 'Он даже меня не замечает.',
 'Тебе это всё нравится?',
 'У него тогда не было с собой денег.',
 'Я его больше не увижу.',
 'Почему она так с ней поступает?',
 'Я хотел бы учиться в Бостоне.',
 'Том и Мэри объявили сегодня о своей пбмолвке.',
 'Том этим сейчас занимается.',
 'Я не могу больше ждать.',
 'Мост очень длинный и высокий.',
 'Том был просто не готов.',
 'Пусть свиньи это едят.',
 'Я ничего не хочу делать.',
 'Ты хотел мне рассказать о свободе?',
 'Что пел Джон на сцене?',
 'Я написал влера письмо.',
 'Тому это тоже не нравится.',
 'Сколько сейчас времени в Париже?',
 'Мы не были готовы.',
 'Они их только что нашли.',
 'Тому 

In [5]:
import os

# Папка, где хранятся ваши txt файлы
data_folder = 'c:\\Users\\warpa\\Synonyms'

# Создадим пустой словарь для хранения данных
synonym_tables = {}

for filename in os.listdir(data_folder):
    if filename.endswith('.txt'):
        with open(os.path.join(data_folder, filename), 'r', encoding='utf-8') as file:
            for line in file:
                # Разделим строку на слово и его синонимы
                word, synonyms = line.split('[')
                word = word.strip(',').lower()
                synonyms = [s.strip(' \'').lower() for s in synonyms.split(']')[0].split(',')]

                # Определим первую букву слова
                first_letter = word[0].lower()

                # Если буква уже есть в таблице, добавим слово и синонимы
                if first_letter in synonym_tables:
                    if word in synonym_tables[first_letter]:
                        synonym_tables[first_letter][word].extend(synonyms)
                    else:
                        synonym_tables[first_letter][word] = synonyms
                else:
                    # В противном случае, создадим новую хэш-таблицу
                    synonym_tables[first_letter] = {word: synonyms}

# Пример использования таблицы для слов на букву 'а'
if 'а' in synonym_tables:
    print(synonym_tables['к'])


{'к бабке ходить не надо': ['понятный'], 'к вашему сведению': ['знай', 'знайте'], 'к верховью': ['вверх'], 'к вечеру': ['вечером'], 'к гадалке ходить не надо': ['понятный'], 'к делу': ['впопад'], 'к добру': ['впрок'], 'к довершению всего': ['вдобавок', 'притом'], 'к довершению того': ['вдобавок'], 'к довершению чего': ['притом'], 'к домашним пенатам': ['домой'], 'к истоку': ['вверх'], 'к лицу': ['идет', 'подобает', 'подходит', 'пригоже', 'прилично', 'пристало', 'приходится'], 'к месту': ['впопад', 'кстати', 'уместно'], 'к несчастью': ['жаль', 'увы'], 'к низу': ['вниз'], 'к примеру': ['в частности', 'взять', 'так', 'хоть', 'например', 'примерно', 'скажем'], 'к примеру сказать': ['взять', 'например', 'примерно', 'скажем', 'так', 'хоть'], 'к родным пенатам': ['домой'], 'к рождеству зарежем': ['кормивший'], 'к руке': ['впопад', 'кстати', 'уместно'], 'к своим пенатам': ['домой'], 'к себе': ['восвояси', 'домой'], 'к середине': ['внутрь', 'вовнутрь'], 'к слову сказать': ['кстати'], 'к сожален

In [ ]:
import pymorphy2
from tqdm import tqdm
import string

morph = pymorphy2.MorphAnalyzer()

def preprocess_sentence(sentence, word_set, synonym_tables):
    sentence = remove_punctuation(sentence)
    sentence = replace_unknown_words(sentence, word_set, synonym_tables)
    sentence = replace_pronouns(sentence)
    sentence = replace_prepositions(sentence)
    sentence = lemmatize_sentence(sentence)
    return sentence

def find_synonyms_in_table(sentence, synonym_tables):
    words = sentence.lower().split()
    synonyms_dict = {}

    for word in words:
        normalized_word = morph.parse(word)[0].normal_form
        letter = normalized_word[0].lower()

        if letter in synonym_tables and normalized_word in synonym_tables[letter]:
            synonyms = synonym_tables[letter][normalized_word]
            synonyms_dict[word] = synonyms

    return synonyms_dict

def compare_sentences_with_synonyms(sentence1, sentence2, synonym_tables):
    synonyms_dict1 = find_synonyms_in_table(sentence1, synonym_tables)
    synonyms_dict2 = find_synonyms_in_table(sentence2, synonym_tables)

    for word1, synonyms1 in synonyms_dict1.items():
        for word2 in sentence2.split():
            if word1 == word2 or any(word2 in synonyms1 for synonym in synonyms1):
                sentence1 = sentence1.replace(word1, word2)

    return sentence1, sentence2

def lemmatize_sentence(sentence):
    return ' '.join([morph.parse(word)[0].normal_form for word in sentence.split()])

def compare_sentences(words1, words2, synonym_tables):
    # Получаем предложения с учетом синонимов
    sentence1, sentence2 = compare_sentences_with_synonyms(words1, words2, synonym_tables)

    lemmas1 = set(sentence1.split())
    lemmas2 = set(sentence2.split())

    common_words = lemmas1 & lemmas2

    common_words_synonyms = set()
    for word in common_words:
        if word in synonym_tables:
            common_words_synonyms.update(synonym_tables[word])

    total_words = lemmas1.union(lemmas2).union(common_words_synonyms)

    similarity = len(common_words) / len(total_words) if len(total_words) > 0 else 0

    if ('не' in lemmas1 and 'не' in lemmas2) or ('не' not in lemmas1 and 'не' not in lemmas2):
        similarity *= 1
    else:
        similarity = similarity - 0.3

    parsed_words1 = [morph.parse(word)[0].normal_form for word in sentence1.split() if morph.parse(word)[0].tag.POS in ('VERB', 'INFN')]
    parsed_words2 = [morph.parse(word)[0].normal_form for word in sentence2.split() if morph.parse(word)[0].tag.POS in ('VERB', 'INFN')]

    if set(parsed_words2) != set(parsed_words1):
        similarity = similarity - 0.3
    else:
        similarity = similarity

    return similarity


all_words_set = set(word for sentence in sentences for word in sentence.split())

similarity_matrix = {}

for i in tqdm(range(len(sentences))):
    for j in range(i+1, len(sentences)):
        preprocessed_sentence_i = preprocess_sentence(sentences[i], all_words_set, synonym_tables)
        preprocessed_sentence_j = preprocess_sentence(sentences[j], all_words_set, synonym_tables)
        similarity = compare_sentences(preprocessed_sentence_i, preprocessed_sentence_j, synonym_tables)
        similarity_matrix[(i, j)] = similarity


for (i, j), similarity in similarity_matrix.items():
    print(f"Сходство между предложением {i + 1} и предложением {j + 1}: {similarity}")


  4%|███▏                                                                             | 16/412 [01:59<48:01,  7.28s/it]

In [ ]:
sim = [(i, j,similarity ) for (i, j), similarity in similarity_matrix.items() if 0.4<similarity<0.5]

In [ ]:
for i, j, similarity in sim:
    sentence1 = sentences[i]
    sentence2 = sentences[j]
    print(f"Похожие предложения (сходство {similarity:.2f}):")
    print(f"{sentence1}")
    print(f"{sentence2}")
    print()